In [1]:
import time
import pandas as pd
from tqdm import tqdm
from nba_api.stats.endpoints import leaguegamefinder, playbyplayv2

In [6]:
# ========== Parameters ==========
SEASON = "2020-21"  # Change to desired season
SEASON_TYPE = "Regular Season"  # Usually "Regular Season" or "Playoffs"
OUTPUT_FILE = f"nba_playbyplay_{SEASON.replace('-', '')}.csv"

# ========== Step 1: Get all games in the season ==========
print(f"Getting all {SEASON_TYPE} games for season {SEASON}...")
game_finder = leaguegamefinder.LeagueGameFinder(
    season_nullable=SEASON,
    season_type_nullable=SEASON_TYPE
)
df_games = game_finder.get_data_frames()[0]
game_ids = df_games["GAME_ID"].unique()

print(f"Found {len(game_ids)} games in {SEASON} {SEASON_TYPE}.")

# ========== Step 2: Loop through games and get play-by-play ==========
all_pbp = []

for game_id in tqdm(game_ids, desc="Downloading play-by-play data"):
    try:
        pbp = playbyplayv2.PlayByPlayV2(game_id=game_id)
        df_pbp = pbp.get_data_frames()[0]
        df_pbp["GAME_ID"] = game_id
        all_pbp.append(df_pbp)
        time.sleep(0.6)  # To avoid hitting API limits
    except Exception as e:
        print(f"Error fetching play-by-play for game {game_id}: {e}")
        time.sleep(1)

# ========== Step 3: Combine and save ==========
if all_pbp:
    df_all_pbp = pd.concat(all_pbp, ignore_index=True)
    df_all_pbp.to_csv(OUTPUT_FILE, index=False)
    print(f"Saved play-by-play data for {len(game_ids)} games to {OUTPUT_FILE}")
else:
    print("No play-by-play data collected.")


Getting all Regular Season games for season 2020-21...
Found 1215 games in 2020-21 Regular Season.


Saved play-by-play data for 1215 games to nba_playbyplay_202021.csv


In [12]:
df_all_pbp.describe()

,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,PERSON1TYPE,PLAYER1_ID,PLAYER1_TEAM_ID,PERSON2TYPE,PLAYER2_ID,PLAYER2_TEAM_ID,PERSON3TYPE,PLAYER3_ID,PLAYER3_TEAM_ID,VIDEO_AVAILABLE_FLAG
count,562044.000000,562044.000000,562044.000000,562044.000000,562039.000000,5.620440e+05,5.162610e+05,562044.000000,5.620440e+05,1.853440e+05,562044.000000,5.620440e+05,1.393300e+04,562044.000000
mean,335.968704,3.960775,17.393261,2.536784,4.281235,9.748095e+07,1.610848e+09,1.485416,3.548857e+05,1.610859e+09,0.231395,2.703762e+05,1.610837e+09,0.728025
std,195.540746,2.705761,30.796621,1.134678,0.932876,3.820050e+08,6.615084e+05,2.137039,6.457574e+05,6.749437e+05,0.757724,1.980940e+07,6.477409e+05,0.444978
min,2.000000,1.000000,0.000000,1.000000,0.000000,0.000000e+00,1.610613e+09,0.000000,0.000000e+00,1.610613e+09,0.000000,0.000000e+00,1.610613e+09,0.000000
25%,167.000000,2.000000,0.000000,2.000000,4.000000,2.034690e+05,1.610613e+09,0.000000,0.000000e+00,1.610613e+09,0.000000,0.000000e+00,1.610613e+09,0.000000
50%,335.000000,4.000000,1.000000,3.000000,4.000000,1.627827e+06,1.610613e+09,0.000000,0.000000e+00,1.610613e+09,0.000000,0.000000e+00,1.610613e+09,1.000000
75%,502.000000,5.000000,12.000000,4.000000,5.000000,1.629631e+06,1.610613e+09,4.000000,2.031450e+05,1.610613e+09,0.000000,0.000000e+00,1.610613e+09,1.000000
max,846.000000,18.000000,110.000000,6.000000,7.000000,1.612710e+09,1.612710e+09,5.000000,1.630466e+06,1.612710e+09,5.000000,1.612710e+09,1.612710e+09,1.000000
